<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [92]:
import numpy as np
import pandas as pd

In [93]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [94]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [95]:
#Lista de términos t1, t2 y t3 de los 3 documentos
t1 = corpus[0].split(" ")
t2 = corpus[1].split(" ")
t3 = corpus[2].split(" ")
print(f'lista de terminos')
print(f't1 :',t1)
print(f't2 :',t2)
print(f't3 :',t3)
# print(t1,t2,t3)
# vect, términos no repetidos de los documentos
print(f'Vector de términos no repetidos :', np.array(set(t1+t2+t3)))


lista de terminos
t1 : ['que', 'dia', 'es', 'hoy']
t2 : ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes']
t3 : ['martes', 'muchas', 'gracias']
Vector de términos no repetidos : {'es', 'muchas', 'el', 'martes', 'dia', 'que', 'de', 'gracias', 'hoy'}


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [96]:
def One_Hot(text):
    t=list()
    for i in range(len(text)):
        t +=text[i].split(" ")
    uni_t = set(t)
    char_to_int = dict((c, i) for i, c in enumerate(uni_t))
    # int_to_char = dict((i, c) for i, c in enumerate(uni_t))
    # print(f'char_to_int :', char_to_int)
    matriz = np.zeros((len(text),len(uni_t)), dtype=int)
    for i in range(len(text)):
        int_encoded = [char_to_int[char] for char in set(text[i].split(" "))]
        # print(f'int_encoded :',int_encoded)
        onehot_encoded = list()
        for j in int_encoded:
	        letter = [0 for _ in range(len(uni_t))]
	        letter[j] = 1
	        onehot_encoded.append(letter)
        # print(f'onehot_encoded :',onehot_encoded)
        one_hot = np.array(onehot_encoded)
        one_hot = one_hot.sum(axis=0).reshape(1, -1)
        # print(f'onehot_spot :',one_hot.shape)
        matriz[i] = one_hot
    # print(f'matriz :',matriz.shape)

    df = pd.DataFrame(matriz)
    df.set_axis(char_to_int, axis=1,inplace=True)
    # print(df.shape)
    return df

In [97]:
print(One_Hot(corpus))

   es  muchas  el  martes  dia  que  de  gracias  hoy
0   1       0   0       0    1    1   0        0    1
1   1       0   1       1    1    0   1        0    1
2   0       1   0       1    0    0   0        1    0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [98]:
def Count_Vect(text):
    #t = text.split(" ")
    a=list()
    for i in range(len(text)):
        a +=corpus[i].split(" ")
    uni_t = set(a)
    char_to_int = dict((c, i) for i, c in enumerate(uni_t))
    matriz = np.zeros((len(text),len(uni_t)), dtype=int)
    df= pd.DataFrame([char_to_int])
    df=df.drop([0],axis=0)
    for j in range(len(text)):
        t = text[j].split(" ")
        count = {}
        for i in t:
            if not i in count:
                count[i] = 1
            else:
                count[i] +=1

        df1 = pd.DataFrame([count])
        df = pd.concat([df, df1], ignore_index=True)


    df = df.fillna(0)

    return df.astype(int)

In [99]:
print(Count_Vect(corpus))

   es  muchas  el  martes  dia  que  de  gracias  hoy
0   1       0   0       0    1    1   0        0    1
1   1       0   1       2    1    0   1        0    1
2   0       1   0       1    0    0   0        1    0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [100]:
def tf_idf(text):
    num = len(text)
    idf =np.log(num/One_Hot(text).sum(axis=0))
    tf = Count_Vect(text)
    tf_idf = pd.DataFrame(tf*idf)
    return tf_idf

In [101]:
print(tf_idf(corpus))

         es    muchas        el    martes       dia       que        de  \
0  0.405465  0.000000  0.000000  0.000000  0.405465  1.098612  0.000000   
1  0.405465  0.000000  1.098612  0.810930  0.405465  0.000000  1.098612   
2  0.000000  1.098612  0.000000  0.405465  0.000000  0.000000  0.000000   

    gracias       hoy  
0  0.000000  0.405465  
1  0.000000  0.405465  
2  1.098612  0.000000  


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [102]:
def comp_docs(text,indice1):
    index = len(text)
    df = tf_idf(text)
    a=np.zeros(index, dtype=float)

    for i in range(index):
        print(f'Text {i}: ',text[i])
        a [i]= cosine_similarity(df.iloc[indice1],df.iloc[i])

    print('Ordenamiento según similitud cos \n',a)
    b=0
    for i in range(index):
        if a[i]>0.99:
            # print(f'Text {0}: ',text[indice1])
            d=text[indice1]
        else:
            if a[i] > b:
                b=a[i]
                # print(f'Text {1}: ',text[i])
                e=text[i]
            else:
                # print(f'Text {2}: ',text[i])
                f=text[i]

    print(f'Text {0}:',d)
    print(f'Text {1}:',e)
    print(f'Text {2}:',f)
    print(np.sort(a))
    return

In [104]:
comp_docs(corpus,1)

Text 0:  que dia es hoy
Text 1:  martes el dia de hoy es martes
Text 2:  martes muchas gracias
Ordenamiento según similitud cos 
 [0.2003419  1.         0.10845712]
Text 0: martes el dia de hoy es martes
Text 1: que dia es hoy
Text 2: martes muchas gracias
[0.10845712 0.2003419  1.        ]
